# __Mutiple Strategy Method__


## First stage

### 1.Data Preparation

Please allocate the data you want to use in proportion to the corresponding folder.Each image needs a corresponding tag file.

In [1]:
train_data_path = r''

val_data_path = r''

In [1]:
import numpy as np

channel = input("Please enter number of channel")
data_all = []


### 2.Image Composition


Make sure that each of your image data contains all bands.

In [ ]:
from astropy.io import fits
import glob
import os

fits_file_list = glob.glob(os.path.join(train_data_path ,'\*.fits'))
assert len(fits_file_list) % channel == 0, 'Please check your dataset'
for i in range(0, len(fits_file_list), channel):
    path = fits_file_list[i][:-7] + '.fits'
    for j in range(i, i+channel):
        data_all.append(fits.getdata(fits_file_list[j]))

    hdu = fits.PrimaryHDU(data_all)
    hdul = fits.HDUList([hdu])
    hdul.writeto(path)


### 3.Modification of pre-training weights

In [ ]:
import torch

pretrained_weights = torch.load('mask_rcnn_swin_tiny_patch4_window7.pth')


# backbone.patch_embed.proj.weight torch.Size([96, 1, 4, 4])
new_pre_trained_weights = torch.empty(size=[64, int(channel), 7, 7])
for i in range(0, channel):
    new_pre_trained_weights[:, i:i+1, :, :] \
    = pretrained_weights['state_dict']['backbone.patch_embed.proj.weight'][:, :, :, :]

pretrained_weights['state_dict']['backbone.patch_embed.proj.weight'] = \
    new_pre_trained_weights['state_dict']['backbone.patch_embed.proj.weight'][:, :, :, :]


torch.save(pretrained_weights, r"E:\xjbx\Swin2\Swin-Transformer-Object-Detection-master\mask_rcnn_swin_multibrand_%d.pth" % channel)


 default_runtime.py

In [ ]:
pre_model = r''
with open('configs/_base_/default_runtime_1.py', 'r')as f:
    content = f.readlines()
    print(content[-3])
    content[-3] = 'load_from = r"'  + str(pre_model) + '"\n'
content
with open('configs/_base_/default_runtime_1.py', 'w')as f:
    f.writelines(content)

### 4.Modify network

Calculation of mean and variance for standardisation of data.

In [6]:
from astropy.io import fits
import numpy as np

# Pick a composite multi-band data
data_path = r'E:\Galaxy_detection\Swin-Transformer-Object-Detection-master\data\coco\train2017_JWST_galaxy\jw02736001001_02105_00001_nrca1_i2d_0.fits'
data = fits.getdata(data_path)
mean = []
std = []
for i in range(0, int(channel)):
    # mean_ = np.mean(data[i, :, :])
    # std_ = np.std(data[i, :, :])
    mean_ = np.mean(data)
    std_ = np.std(data)
    mean.append(mean_)
    std.append(std_)
    print(mean_, std_)
print(mean, std)


0.15901898 0.025180187
[0.15901898] [0.025180187]


In coco_instance.py, we modify the mean and std.

In [7]:
# 修改 47行
with open('configs/_base_/datasets/coco_instance.py', 'r') as f:
    content = f.readlines()
    print(content[4])
    print(content[5])
    mean_str = '    mean=' + str(mean) + ',\n'
    std_str =  '    std=' + str(std) + ',\n'
    content[4] = mean_str
    content[5] = std_str
content
with open('configs/_base_/datasets/coco_instance.py', 'w') as f:
    f.writelines(content)
    print(True)


    mean=[0.05358604],

    std=[0.9776479],



 In mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py, we modify the mean and std.

In [10]:
with open('configs/swin/mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py', 'r')as f:
    content = f.readlines()
    mean_str = '    mean=' + str(mean) + ',\n'
    std_str =  '    std=' + str(std) + ',\n'
    content[22] = mean_str
    content[23] = std_str
with open('configs/swin/mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py', 'w')as f:
    f.writelines(content)
    print(True)

 In mask_rcnn_swin_fpn.py, we modify the input channels.

In [12]:
with open('configs/_base_/models/mask_rcnn_swin_fpn.py' , 'r') as f:

    content = f.readlines()
    in_chans_str = '        in_chans=%s,\n'%channel,
    content[7] = in_chans_str
content[7] = content[7][0]
content
with open('configs/_base_/models/mask_rcnn_swin_fpn.py' , 'w') as f:
    f.writelines(content)
    print(True)

True


### 5.Mutil-Step Method

In [15]:
from tools import train

train.main()

usage: ipykernel_launcher.py [-h] [--work-dir WORK_DIR]
                             [--resume-from RESUME_FROM] [--no-validate]
                             [--gpus GPUS | --gpu-ids GPU_IDS [GPU_IDS ...]]
                             [--seed SEED] [--deterministic]
                             [--options OPTIONS [OPTIONS ...]]
                             [--cfg-options CFG_OPTIONS [CFG_OPTIONS ...]]
                             [--launcher {none,pytorch,slurm,mpi}]
                             [--local_rank LOCAL_RANK]
                             config
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

Mask Galaxy

In [ ]:
from mmdet.apis import inference_detector, init_detector

configs = r'configs/swin/mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py'
# Saving catalogue for training weights
checkpoints = r''
devices = 'cuda:0'

model = init_detector(configs, checkpoints, device=devices)
# path of training set
fits_path = r' '
for f_path in fits_path:
    base_new_path = r'E:\xjbx\Swin2\Swin-Transformer-Object-Detection-master\data\coco\val2017_multi'
    result = inference_detector(model, f_path)
    try:
        bbox = result[0][0]
        img_data = fits.getdata(f_path)
        img_data = np.flip(img_data, 0)
        for i in range(0, img_data.shape[0]):
            for j in range(0, len(result)):
                img_data[i][int(bbox[j][1]): int(bbox[j][3]), int(bbox[j][0]): int(bbox[j][2])] = 0
        new_fits_path = f_path[-24:-5] + '_mask.fits'
        new_fits_path = os.path.join(base_new_path, new_fits_path)
        print(new_fits_path)
        hdu = fits.PrimaryHDU(img_data)
        hdul = fits.HDUList([hdu])
        hdul.writeto(new_fits_path)
    except:
        with open('none_val_result.txt', 'a+') as f:
            f.writelines(f_path + '\n')

## Second Stage

### 1.Data Prepration

In [1]:
train_data_path = r''

val_data_path = r''


### 2.Modify Network

In [ ]:
import torch
pretrained_weights = torch.load('faster_rcnn_r50_fpn_mstrain_3x_coco_20210524_110822-e10bd31c.pth')


# backbone.patch_embed.proj.weight torch.Size([96, 1, 4, 4])
new_pre_trained_weights = torch.empty(size=[64, int(channel), 7, 7])
for i in range(0, channel):
    new_pre_trained_weights[:, i:i+1, :, :] \
    = pretrained_weights['state_dict']['backbone.conv1.weight'][:, 0:1, :, :]

pretrained_weights['state_dict']['backbone.patch_embed.proj.weight'] = new_pre_trained_weights


torch.save(pretrained_weights, r"E:\xjbx\Swin2\Swin-Transformer-Object-Detection-master\faster_rcnn_r50_fpn_mstrain_multibrand_%d.pth" % channel)

In [ ]:
with open('configs/_base_/models/fast_rcnn_r50_fpn.py' , 'r') as f:

    content = f.readlines()
    in_chans_str = '        in_channels=%s,\n'%channel,
    content[6] = in_chans_str
content[7] = content[7][0]
content
with open('configs/_base_/models/fast_rcnn_r50_fpn.py' , 'w') as f:
    f.writelines(content)

In [ ]:
with open('configs/_base_/datasets/coco_detection.py', 'r') as f:
    content = f.readlines()
    print(content[4])
    print(content[5])
    mean_str = '    mean=' + str(mean) + ',\n'
    std_str =  '    std=' + str(std) + ',\n'
    content[4] = mean_str
    content[5] = std_str
content
with open('configs/_base_/datasets/coco_detection.py', 'w') as f:
    f.writelines(content)

### 3.Training

In [ ]:
from tools import train

train.main()

### 4.Demo

In [2]:
from mmdet.apis import inference_detector, init_detector

configs = r'configs/faster_rcnn/faster_rcnn_r50_fpn_soft_nms_1x_coco.py'
# Saving catalogue for training weights
checkpoints = r''
devices = 'cuda:0'
fits_path = r''
model = init_detector(configs, checkpoints, device=devices)
for f_path in fits_path:
    result = inference_detector(model, f_path)

SyntaxError: invalid syntax (955044061.py, line 9)

# JWST
JWST has just made public a new dataset that detects new astronomical targets, and we will use the model to make inferences about that type of data.

In [ ]:
from mmdet.apis import inference_detector, init_detector

JWST_PATH = r'E:\browse download\google chrome\jw02736-o001_t001_nircam_clear-f200w_i2d.fits'

configs = r'configs/swin/mask_rcnn_swin_tiny_patch4_window7_mstrain_480-800_adamw_3x_coco.py'
# Saving catalogue for training weights
checkpoints = r""
devices = 'cuda:0'
model = init_detector(configs, checkpoints, device=devices)
result = inference_detector(model, JWST_PATH)